In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
import datetime as dt
from netCDF4 import num2date

In [ ]:
#FiRST OF ALL GETTING THE DATE COLUMN FROM THE NETCDF FILE AND THIS DATE AS THE COLUMN NAME IN EXTRACTED DATASETS

In [ ]:
nc1 = nc.Dataset('Soil_upscaled.nc')#I used the soil netcdf as it matches every other dataset

In [ ]:
times = nc1.variables['time'][3:216]

In [ ]:
units = nc1.variables['time'].units

In [ ]:
dtimes = nc.num2date(times, units = units)

In [ ]:
date_array = [dt.datetime(date.year, date.month, date.day) for date in dtimes]
date_array

In [ ]:
Surface_water = pd.read_csv('surface_runoff.csv')
Surface_water_subset = Surface_water.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Surface_water_subset.set_axis(date_array, axis ='columns', inplace=True)#Replacing column name with the dates
SWA = Surface_water_subset.multiply(1000)
SWA

In [ ]:
#SWA.to_csv('SWA.csv', index=False)

In [ ]:
lat_lon_values = Surface_water.iloc[:, 0:2]

In [ ]:
lat_lon_values

In [ ]:
SWA = pd.concat([lat_lon_values.iloc[:, 0:2], SWA], axis=1)
SWA

In [ ]:
SWA.to_csv('Surface_water_ERA5.csv', index=False)

In [ ]:
Snow_water = pd.read_csv('Snow_water.csv')
Snow_water_subset = Snow_water.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Snow_water_subset.set_axis(date_array, axis ='columns', inplace=True)
SWE = Snow_water_subset.multiply(1000)
SWE = pd.concat([lat_lon_values.iloc[:, 0:2], SWE], axis=1)
SWE

In [ ]:
SWE.to_csv('Snow_water_ERA5.csv', index=False)

In [ ]:
Soil_src = pd.read_csv('Soil_src.csv')
Soil_src_subset = Soil_src.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Soil_src_subset.set_axis(date_array, axis ='columns', inplace=True)
Src = Soil_src_subset.multiply(1000)
Src

In [ ]:
Soil_swl1 = pd.read_csv('Soil_swl1.csv')
Soil_swl1_subset = Soil_swl1.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Soil_swl1_subset.set_axis(date_array, axis ='columns', inplace=True)
Swl1 = Soil_swl1_subset.multiply(70) #Volume of water in soil layer 1 (0 - 7 cm) 
Swl1

In [ ]:
Soil_swl2 = pd.read_csv('Soil_swl2.csv')
Soil_swl2_subset = Soil_swl2.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Soil_swl2_subset.set_axis(date_array, axis ='columns', inplace=True)
Swl2 = Soil_swl2_subset.multiply(210) #Volume of water in soil layer 2 (7 -28 cm) of the ECMWF Integrated Forecasting System.
Swl2

In [ ]:
Soil_swl3 = pd.read_csv('Soil_swl3.csv')
Soil_swl3_subset = Soil_swl3.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Soil_swl3_subset.set_axis(date_array, axis ='columns', inplace=True)
Swl3 = Soil_swl3_subset.multiply(720)#Volume of water in soil layer 3 (28-100 cm) of the ECMWF Integrated Forecasting System.
Swl3

In [ ]:
Soil_swl4 = pd.read_csv('Soil_swl4.csv')
Soil_swl4_subset = Soil_swl4.iloc[:, 2:215]#Subsetting from column 2-145(variables excluding lat-lon values)
Soil_swl4_subset.set_axis(date_array, axis ='columns', inplace=True)
Swl4 = Soil_swl4_subset.multiply(1890)#Volume of water in soil layer 4 (100-289 cm) of the ECMWF Integrated Forecasting System.
Swl4

In [ ]:
Total_soil = Src.add(Swl1, fill_value=0).add(Swl2,fill_value=0).add(Swl3, fill_value=0).add(Swl4,fill_value=0)
Total_soil

In [ ]:
Total_soil
Total_soil_with_coord = pd.concat([lat_lon_values.iloc[:, 0:2], Total_soil], axis=1)
Total_soil_with_coord

In [ ]:
Total_soil_with_coord.to_csv('Total_soil_ERA5.csv', index=False)

In [ ]:
Total_Soil_Snow_Surface = Total_soil.add(SWE, fill_value=0).add(SWA,fill_value=0)

In [ ]:
Total_Soil_Snow_Surface

In [ ]:
lat_lon_values = Surface_water.iloc[:, 0:2]

In [ ]:
lat_lon_values

In [ ]:
Total_Soil_Snow_Surface = pd.concat([lat_lon_values.iloc[:, 0:2], Total_Soil_Snow_Surface], axis=1)

In [ ]:
Total_Soil_Snow_Surface.columns =  Total_Soil_Snow_Surface.columns.astype(str)#CONVERTED THE COLUMS TO STRING FROM DATE FORMAT, ELSE THIS WOULD NOT HAVE WORKED.

In [ ]:
Total_Soil_Snow_Surface

In [ ]:
#=================HYDROLOGICAL BALANCING PROPER ΔTWS=ΔSM+ΔSWE+ΔSWA+ΔGWS =================================================

In [ ]:
TWS= pd.read_csv('lwe_thickness_retimed.csv')
TWS

In [ ]:
TWS_mm = TWS.multiply(10)
TWS_mm


In [ ]:
GWS = TWS_mm.subtract(Total_Soil_Snow_Surface, fill_value=0)

In [ ]:
GWS = GWS.drop(columns=['lat', 'lon']) #I dropped the latitude and longitude values because the substraction affected the values
GWS

In [ ]:
GWS = pd.concat([lat_lon_values, GWS], axis=1) #Added fresh Lat-Long values to the GWS Series

In [ ]:
GWS

In [ ]:
GWS.to_csv('GWS.csv', index=False)

In [ ]:
#=================HYDROLOGICAL BALANCING ΔTWS=ΔSWE+ΔSWA+ΔGWS (No soil moisture)==============================================

In [ ]:
TWS= pd.read_csv('lwe_thickness_retimed.csv')
TWS

In [ ]:
TWS_mm = TWS.multiply(10) #converting from centimeter to millimeter
TWS_mm

In [ ]:
SWE_SWA = SWE.add(SWA,fill_value=0)
SWE_SWA = pd.concat([lat_lon_values.iloc[:, 0:2], SWE_SWA], axis=1)
SWE_SWA.columns =SWE_SWA.columns.astype(str)#CONVERTED THE COLUMS TO STRING FROM DATE FORMAT, ELSE THIS WOULD NOT HAVE WORKED.
SWE_SWA

In [ ]:
GWS_no_soil = TWS_mm.subtract(SWE_SWA, fill_value=0)
GWS_no_soil

In [ ]:
GWS_no_soil = GWS_no_soil.drop(columns=['lat', 'lon']) #I dropped the latitude and longitude values because the substraction affected the values

In [ ]:
GWS_no_soil = pd.concat([lat_lon_values, GWS_no_soil], axis=1) #Added fresh Lat-Long values to the GWS Series

In [ ]:
GWS_no_soil

In [ ]:
GWS_no_soil.to_csv('GWS_no_soil.csv', index=False)